In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import torch
print("pytorch ver . ",torch.__version__)
print("Is CUDA available?",torch.cuda.is_available())
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.utils.data as Data
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import xarray as xr
import os
os.environ['R_HOME'] = '/home/luhung3080/miniconda3/envs/chou/lib/R'
from rpy2.robjects import r, numpy2ri
numpy2ri.activate()
from rpy2.robjects.packages import importr
sinkr = importr('sinkr')
import Nio
print ("pynio ver . ",Nio.__version__)

pytorch ver .  1.11.0+cu113
Is CUDA available? True
pynio ver .  1.5.5


# Y = b0 + b1X1 + b2X2

## Data 

In [86]:
data=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/data_final.csv')
data

,SITE_ID,date,FCST_TIME,TAU,pm25_cal,pm25_obs
0,EPA001,2020-02-24 08:00:00,2020-02-24 09:00:00,1,4.9510,10.0
1,EPA001,2020-02-24 08:00:00,2020-02-24 10:00:00,2,4.4674,13.0
2,EPA001,2020-02-24 08:00:00,2020-02-24 11:00:00,3,4.6159,11.0
3,EPA001,2020-02-24 08:00:00,2020-02-24 12:00:00,4,3.9937,11.0
4,EPA001,2020-02-24 08:00:00,2020-02-24 13:00:00,5,3.9602,9.0
...,...,...,...,...,...,...
3092755,EPA080,2021-10-30 08:00:00,2021-11-02 04:00:00,68,3.6190,4.0
3092756,EPA080,2021-10-30 08:00:00,2021-11-02 05:00:00,69,3.7908,7.0
3092757,EPA080,2021-10-30 08:00:00,2021-11-02 06:00:00,70,4.0454,7.0
3092758,EPA080,2021-10-30 08:00:00,2021-11-02 07:00:00,71,3.9015,4.0


## 一開始就把x1 x2分開讀取，補完再用x1推y

In [87]:
x1=np.zeros([604,1704])
x2=np.zeros([604,5112])

#x1 (0th~1703th column as x)
for i in range (0,604):
    for j in range (0,71):
        a=np.array(data['pm25_obs'][5112*i+72*j:5112*i+72*j+24])
        for k in range (0,24):
            x1[i][j*24+k]=a[k]

#x2 (1704th~8519th column as x)
for i in range (1,604):
    b=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    for j in range(0,5112):
        x2[i-1][j]=b[j]
        
print(np.shape(x1))
print(np.shape(x2))

(604, 1704)
(604, 5112)


In [88]:
x1Restruct_Fun=x1
x2Restruct_Fun=x2
print(np.shape(x1Restruct_Fun))
print(np.shape(x2Restruct_Fun))

(604, 1704)
(604, 5112)


In [89]:
YRestruct_Fun=np.zeros([601,5112])
for j in range (0,71):
    for i in range(0,601):
        YRestruct_Fun[i][72*j:72*j+24]=x1Restruct_Fun[1+i][24*j:24*j+24]
        YRestruct_Fun[i][72*j+24:72*j+48]=x1Restruct_Fun[1+i+1][24*j:24*j+24]
        YRestruct_Fun[i][72*j+48:72*j+72]=x1Restruct_Fun[1+i+2][24*j:24*j+24]
print(np.shape(YRestruct_Fun))

(601, 5112)


In [90]:
XRestruct_Fun=np.zeros([601,6816])
for i in range (0,601):
    for j in range (0,1704):
        XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
    for j in range (1704,6816):
        XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]
print(np.shape(XRestruct_Fun))

(601, 6816)


In [91]:
Xhat=np.zeros([540,6816])
Yhat=np.zeros([540,5112])
for i in range (0,540):
    for j in range (0,5112):
        Xhat[i][j]=XRestruct_Fun[i][j]
        Yhat[i][j]=YRestruct_Fun[i][j]
print('Xhat')
#print(Xhat)
print(np.shape(Xhat))
print('Yhat')
#print(Yhat)
print(np.shape(Yhat))

Xhat
(540, 6816)
Yhat
(540, 5112)


In [92]:
print(np.linalg.matrix_rank(Xhat))
print(np.linalg.matrix_rank(Yhat))

540
540


## sparse x1 x2

In [93]:
x1=np.zeros([540,1704])
x2=np.zeros([540,5112])
for i in range (0,540):
    for j in range (0,1704):
        x1[i][j]=Xhat[i][j]
    for j in range (0,5112):
        x2[i][j]=Xhat[i][1704+j]
print(np.shape(x1))
print(np.shape(x2))

(540, 1704)
(540, 5112)


## expend x1 x2

In [94]:
new_x=np.zeros([540*72,6816]) #38880
new_y=np.zeros([540*72,5112]) #38880
#new_x : 0th~1703th column
for i in range (0,72):
    for j in range (0,540):
        for k in range (0,1704):
            new_x[540*i+j][k]=x1[j][k]
#new_x : 1704th~8519th column
for i in range (0,72):
    for j in range (0,540):
        for k in range (0+i,5040+1+i,72): #5040=5112-72
            new_x[540*i+j][1704+k]=x2[j][k]
#new_y
for i in range (0,72):
    for j in range (0,540):
        for k in range (0,5112):
            new_y[540*i+j][k]=Yhat[j][k]         

XXhat=new_x
YYhat=new_y
print(np.shape(new_x))
print(np.shape(new_y))
print(np.shape(XXhat))
print(np.shape(YYhat))

(38880, 6816)
(38880, 5112)
(38880, 6816)
(38880, 5112)


In [53]:
print(np.linalg.matrix_rank(new_x))
print(np.linalg.matrix_rank(new_y))

KeyboardInterrupt: 

## PCA95%

In [65]:
pca95 = PCA(n_components=0.95)
pca95.fit(new_x)

PCA(n_components=0.95)

In [66]:
x_pca95=pca95.transform(new_x)
print(x_pca95.shape)
print(np.linalg.matrix_rank(x_pca95))

(38880, 217)
217


### LinearRegression

In [67]:
model = LinearRegression()
model.fit(x_pca95, new_y)
print(np.shape(model.coef_))

(5112, 217)


In [68]:
fit_y=model.predict(x_pca95)
print(fit_y.shape)

(38880, 5112)


In [69]:
fit95=np.zeros([540,5112])
for i in range (0,72):
    for j in range (0,540):
        for k in range (0,71):
            fit95[j][72*k+i]=fit_y[540*i+j][72*k+i]
print(np.shape(fit95))

(540, 5112)


In [70]:
fit95_mean=np.zeros([540,5112])
for i in range (0,540):
    for j in range (0,5112):
        for k in range (0,72):
            fit95_mean[i][j]=fit95_mean[i][j]+fit_y[540*k+i][j]/72
print(np.shape(fit95_mean))

(540, 5112)


### compare(training data)

In [71]:
fit95_test=fit95
fit95_mean_test=fit95_mean

In [72]:
data0=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/Data/newdata/result.csv')
dd = data0.drop(data0[data0['TAU'] == 0].index)

In [73]:
"""
old data
"""
old_x=np.zeros([601,5112])
old_y=np.zeros([601,5112])
#data_cal
for i in range (1,602):
    a=np.array(dd['VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
    old_x[i-1]=a
#data_obs
for i in range (0,601):
    a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
        if a[j]=='A' :
            a[j]=np.nan
        if a[j]=='*' :
            a[j]=np.nan
        if a[j]=='x' :
            a[j]=np.nan  
    old_y[i-1]=a
print(np.shape(old_x))
print(np.shape(old_y))
print(np.shape(fit95_test))
print(np.shape(fit95_mean_test))

(601, 5112)
(601, 5112)
(540, 5112)
(540, 5112)


In [74]:
old_x1=np.zeros([540,5112])
old_y1=np.zeros([540,5112])
fit_fx1=np.zeros([540,5112])
fit_fx1mean=np.zeros([540,5112])
for i in range (0,540):
    for j in range (0,5112):
        old_x1[i][j]=old_x[i][j]
        old_y1[i][j]=old_y[i][j]
        fit_fx1[i][j]=fit95_test[i][j]
        fit_fx1mean[i][j]=fit95_mean_test[i][j]
before1=abs(old_x1-old_y1)
after1=abs(fit_fx1-old_y1)
after_mean1=abs(fit_fx1mean-old_y1)
before_error1=np.nanmean(before1)
after_error1=np.nanmean(after1)
after_error_mean1=np.nanmean(after_mean1)

print("January to August (training data set) before_error (MADE) of 95 : %f" %before_error1)
print("January to August (training data set) after_error (MADE) of 95  %f" %after_error1)
print("January to August (training data set) after_error (MADE) of 95_mean : %f" %after_error_mean1)

January to August (training data set) before_error (MADE) of 95 : 7.174635
January to August (training data set) after_error (MADE) of 95  4.506647
January to August (training data set) after_error (MADE) of 95_mean : 4.512997


### compare(testing data)

In [75]:
"""
generate new cal
"""
x1=np.zeros([604,1704])
x2=np.zeros([604,5112])

#x1 (0th~1703th column as x)
for i in range (0,604):
    for j in range (0,71):
        a=np.array(data['pm25_obs'][5112*i+72*j:5112*i+72*j+24])
        for k in range (0,24):
            x1[i][j*24+k]=a[k]

#x2 (1704th~6816th column as x)
for i in range (1,604):
    b=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    for j in range(0,5112):
        x2[i-1][j]=b[j]

x1Restruct_Fun=x1
x2Restruct_Fun=x2
XRestruct_Fun=np.zeros([601,6816])
for i in range (0,601):
    for j in range (0,1704):
        XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
    for j in range (1704,6816):
        XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]
print(np.shape(XRestruct_Fun))
#(601,6816)
####################################################
xf = np.zeros([61,6816])
for i in range (0,61):
    for j in range(0,6816):
        xf[i][j] = XRestruct_Fun[i+540][j]

(601, 6816)


In [76]:
x1 = np.zeros([61,1704])
x2 = np.zeros([61,5112])
for i in range (0,61):
    for j in range (0,1704):
        x1[i][j] = xf[i][j]
    for j in range (1704,6816):
        x2[i][j-1704] = xf[i][j]

new_x=np.zeros([61*72,6816]) #4392
#new_x : 0th~1703th column
for i in range (0,72):
    for j in range (0,61):
        for k in range (0,1704):
            new_x[61*i+j][k]=x1[j][k]
#new_x : 1704th~8519th column
for i in range (0,72):
    for j in range (0,61):
        for k in range (0+i,5040+1+i,72):  #5040=5112-72
            new_x[61*i+j][1704+k]=x2[j][k]
            
print(np.shape(new_x))
# print(np.linalg.matrix_rank(new_x))

new_xcal=pca95.transform(new_x)
fit_y=model.predict(new_xcal)
print(np.shape(fit_y))

fit95=np.zeros([61,5112])
for i in range (0,72):
    for j in range (0,61):
        for k in range (0,71):
            fit95[j][72*k+i]=fit_y[61*i+j][72*k+i]
print(np.shape(fit95))

fit95_mean=np.zeros([61,5112])
for i in range (0,61):
    for j in range (0,5112):
        for k in range (0,72):
            fit95_mean[i][j]=fit95_mean[i][j]+fit_y[61*k+i][j]/72
print(np.shape(fit95_mean))

(4392, 6816)
(4392, 5112)
(61, 5112)
(61, 5112)


In [77]:
fit95_test=fit95
fit95_mean_test=fit95_mean

In [78]:
"""
old data
"""
old_x=np.zeros([601,5112])
old_y=np.zeros([601,5112])
#data_cal
for i in range (1,602):
    a=np.array(dd['VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
    old_x[i-1]=a
#data_obs
for i in range (1,602):
    a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
        if a[j]=='A' :
            a[j]=np.nan
        if a[j]=='*' :
            a[j]=np.nan
        if a[j]=='x' :
            a[j]=np.nan  
    old_y[i-1]=a

old_x2=np.zeros([61,5112])
old_y2=np.zeros([61,5112])
fit_fx2=np.zeros([61,5112])
fit_fx2mean=np.zeros([61,5112])
for i in range (0,61):
    for j in range (0,5112):
        old_x2[i][j]=old_x[i+540][j]
        old_y2[i][j]=old_y[i+540][j]
        fit_fx2[i][j]=fit95_test[i][j]
        fit_fx2mean[i][j]=fit95_mean_test[i][j]
before2=abs(old_x2-old_y2)
after2=abs(fit_fx2-old_y2)
after_mean2=abs(fit_fx2mean-old_y2)

before_error2=np.nanmean(before2)
after_error2=np.nanmean(after2)
after_error_mean2=np.nanmean(after_mean2)
print("September (testing data set) before_error (MADE) : %f" %before_error2)
print("September (testing data set) after_error (MADE) : %f" %after_error2)
print("September (testing data set) after_error (MADE) of 95_mean: %f" %after_error_mean2)

September (testing data set) before_error (MADE) : 6.989377
September (testing data set) after_error (MADE) : 6.122959
September (testing data set) after_error (MADE) of 95_mean: 6.110281


# loop test

In [95]:
per = np.zeros(99)
for i in range (0,99):
    per[i]=0.01*(i+1)
print(per)
per_error1 = np.zeros(99)
per_error2 = np.zeros(99)

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28
 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42
 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56
 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7
 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84
 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98
 0.99]


In [96]:
new_x.shape

(38880, 6816)

In [97]:
for z in range (0,99):
    print(z+1)
    pca95 = PCA(n_components=per[z])
    pca95.fit(new_x)
    x_pca95=pca95.transform(new_x)
    print(x_pca95.shape)
    model = LinearRegression()
    model.fit(x_pca95, new_y)
    print(np.shape(model.coef_))
    fit_y=model.predict(x_pca95)
    print(fit_y.shape)
    """
    generate new cal
    """
    x1=np.zeros([604,1704])
    x2=np.zeros([604,5112])

    #x1 (0th~1703th column as x)
    for i in range (0,604):
        for j in range (0,71):
            a=np.array(data['pm25_obs'][5112*i+72*j:5112*i+72*j+24])
            for k in range (0,24):
                x1[i][j*24+k]=a[k]

    #x2 (1704th~6816th column as x)
    for i in range (1,604):
        b=np.array(data['pm25_cal'][5112*i:5112*i+5112])
        for j in range(0,5112):
            x2[i-1][j]=b[j]

    x1Restruct_Fun=x1
    x2Restruct_Fun=x2
    XRestruct_Fun=np.zeros([601,6816])
    for i in range (0,601):
        for j in range (0,1704):
            XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
        for j in range (1704,6816):
            XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]
    print(np.shape(XRestruct_Fun))
    #(601,6816)
    ####################################################
    xf = np.zeros([61,6816])
    for i in range (0,61):
        for j in range(0,6816):
            xf[i][j] = XRestruct_Fun[i+540][j]

    x1 = np.zeros([61,1704])
    x2 = np.zeros([61,5112])
    for i in range (0,61):
        for j in range (0,1704):
            x1[i][j] = xf[i][j]
        for j in range (1704,6816):
            x2[i][j-1704] = xf[i][j]

    new_exx=np.zeros([61*72,6816]) 
    #new_x : 0th~1703th column
    for i in range (0,72):
        for j in range (0,61):
            for k in range (0,1704):
                new_exx[61*i+j][k]=x1[j][k]
    #new_x : 1704th~8519th column
    for i in range (0,72):
        for j in range (0,61):
            for k in range (0+i,5040+1+i,72):  #5040=5112-72
                new_exx[61*i+j][1704+k]=x2[j][k]

    print(np.shape(new_exx))
    # print(np.linalg.matrix_rank(new_x))

    new_xcal=pca95.transform(new_exx)
    fit_y=model.predict(new_xcal)
    print(np.shape(fit_y))

    fit95=np.zeros([61,5112])
    for i in range (0,72):
        for j in range (0,61):
            for k in range (0,71):
                fit95[j][72*k+i]=fit_y[61*i+j][72*k+i]
    print(np.shape(fit95))

    fit95_mean=np.zeros([61,5112])
    for i in range (0,61):
        for j in range (0,5112):
            for k in range (0,72):
                fit95_mean[i][j]=fit95_mean[i][j]+fit_y[61*k+i][j]/72
    print(np.shape(fit95_mean))

    fit95_test=fit95
    fit95_mean_test=fit95_mean

    """
    old data
    """
    old_x=np.zeros([601,5112])
    old_y=np.zeros([601,5112])
    #data_cal
    for i in range (1,602):
        a=np.array(dd['VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
        old_x[i-1]=a
    #data_obs
    for i in range (1,602):
        a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
            if a[j]=='A' :
                a[j]=np.nan
            if a[j]=='*' :
                a[j]=np.nan
            if a[j]=='x' :
                a[j]=np.nan  
        old_y[i-1]=a

    old_x2=np.zeros([61,5112])
    old_y2=np.zeros([61,5112])
    fit_fx2=np.zeros([61,5112])
    fit_fx2mean=np.zeros([61,5112])
    for i in range (0,61):
        for j in range (0,5112):
            old_x2[i][j]=old_x[i+540][j]
            old_y2[i][j]=old_y[i+540][j]
            fit_fx2[i][j]=fit95_test[i][j]
            fit_fx2mean[i][j]=fit95_mean_test[i][j]
    before2=abs(old_x2-old_y2)
    after2=abs(fit_fx2-old_y2)
    after_mean2=abs(fit_fx2mean-old_y2)

    before_error2=np.nanmean(before2)
    after_error2=np.nanmean(after2)
    after_error_mean2=np.nanmean(after_mean2)
    print("September (testing data set) before_error (MADE) : %f" %before_error2)
    print("September (testing data set) after_error (MADE) : %f" %after_error2)
    print("September (testing data set) after_error (MADE) of 95_mean: %f" %after_error_mean2)
    per_error1[z] = after_error2
    per_error2[z] = after_error_mean2

1
(38880, 1)
(5112, 1)
(38880, 5112)
(601, 6816)
(4392, 6816)
(4392, 5112)
(61, 5112)
(61, 5112)
September (testing data set) before_error (MADE) : 6.989377
September (testing data set) after_error (MADE) : 5.953304
September (testing data set) after_error (MADE) of 95_mean: 5.953342
2
(38880, 1)
(5112, 1)
(38880, 5112)
(601, 6816)
(4392, 6816)
(4392, 5112)
(61, 5112)
(61, 5112)
September (testing data set) before_error (MADE) : 6.989377
September (testing data set) after_error (MADE) : 5.953304
September (testing data set) after_error (MADE) of 95_mean: 5.953342
3


KeyboardInterrupt: 

In [ ]:
print(per_error1)
print(per_error2)

In [ ]:
print(0.01*(np.argmin(per_error1)+1))
print(min(per_error1))

In [ ]:
print(0.01*(np.argmin(per_error2)+1))
print(min(per_error2))